### Multilayer Perceptron

In [7]:
import torch
import torchvision
import torch.nn as nn
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

import numpy as np
print('pytorch version:',torch.__version__,'\ntorchvision version: ',torchvision.__version__,'\nnumpy version:' ,np.__version__)

pytorch version: 1.5.0 
torchvision version:  0.6.0.dev20200319 
numpy version: 1.18.2


#### Settings

In [8]:
# model runs on  GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
random_seed = 21
learning_rate = 1e-3 
num_epochs = 10
batch_size = 64

torch.manual_seed(random_seed)

# Architecture
num_features = 784
num_hidden = 200
num_classes = 10

#### Dataset: MNIST

In [9]:
train_dataset = datasets.MNIST(root='data', 
                               train=True, 
                               transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]),
                               download=True)

test_dataset = datasets.MNIST(root='data', 
                              train=False, 
                              transform=transforms.Compose([
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.1307,), (0.3081,))
                                ]))


train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size, 
                         shuffle=False)

# Checking the dataset
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break
    

Image batch dimensions: torch.Size([64, 1, 28, 28])
Image label dimensions: torch.Size([64])


#### Define model

In [10]:
class MultilayerPerceptron(nn.Module):

    def __init__(self, num_features, num_classes):
        super(MultilayerPerceptron, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(num_features, num_hidden),
            nn.Linear(num_hidden, num_hidden),
            nn.Linear(num_hidden, num_classes)
        )

    def forward(self, x):
        x = self.model(x)

        return x

#### Init model, define optimizer and loss function

In [11]:
model = MultilayerPerceptron(num_features=num_features,
                             num_classes=num_classes)
model = model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

#### Training model

In [12]:
for epoch in range(num_epochs):

    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        data = data.view(-1, 28*28)
        
        # forward
        logits = model(data)
        loss = criteon(logits, target)

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
    
    test_loss = 0
    correct = 0 
    model.eval()
    with torch.no_grad():
        # test
        total_correct = 0
        total_num = 0
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.view(-1, 28*28)

            logits = model(data)
            test_loss += criteon(logits, target).item()

            pred = logits.data.max(1)[1]
            correct += pred.eq(target.data).sum()

        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.278663
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.095090
Train Epoch: 0 [12800/60000 (21%)]	Loss: 1.861798
Train Epoch: 0 [19200/60000 (32%)]	Loss: 1.728410
Train Epoch: 0 [25600/60000 (43%)]	Loss: 1.576093
Train Epoch: 0 [32000/60000 (53%)]	Loss: 1.514901
Train Epoch: 0 [38400/60000 (64%)]	Loss: 1.284239
Train Epoch: 0 [44800/60000 (75%)]	Loss: 1.185161
Train Epoch: 0 [51200/60000 (85%)]	Loss: 1.156796
Train Epoch: 0 [57600/60000 (96%)]	Loss: 1.101600

Test set: Average loss: 0.0155, Accuracy: 8043/10000 (80%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.028768
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.948873
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.017708
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.793532
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.895121
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.670167
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.680979
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.617230
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.672858
T